In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../../data/opening_scores.csv')

In [3]:
df

,user_id,opening_id,score
0,0db6b893-14aa-4ae5-8e57-692ca7943ab5,d569ec70-f3fa-421c-88db-0773b26a2475,4
1,0db6b893-14aa-4ae5-8e57-692ca7943ab5,1067c159-be01-4f89-a1cf-a32cdd67bb72,4
2,90bab470-e5d4-4d93-bd0a-15c51918f172,2bc22ae6-ac3d-4aa5-82a4-dd55b8d8709b,4
3,90bab470-e5d4-4d93-bd0a-15c51918f172,d569ec70-f3fa-421c-88db-0773b26a2475,4
4,90bab470-e5d4-4d93-bd0a-15c51918f172,1067c159-be01-4f89-a1cf-a32cdd67bb72,4
5,5f5a90e2-6601-480c-bc08-7592e06beacd,2bc22ae6-ac3d-4aa5-82a4-dd55b8d8709b,4


In [4]:
from sklearn.preprocessing import LabelEncoder

user_le = LabelEncoder()
df['enc_user_id'] = user_le.fit_transform(df['user_id'])

opening_le = LabelEncoder()
df['enc_opening_id'] = opening_le.fit_transform(df['opening_id'])

In [5]:
X = df[['enc_user_id', 'enc_opening_id']]
y = df['score']

In [6]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input,Dense,Embedding,Flatten,Input,concatenate,Add
from keras.regularizers import l2

In [7]:
num_users = df['enc_user_id'].nunique()
num_openings = df['enc_opening_id'].nunique()
k = 50
l2_lambda = 0.001

In [8]:
u_input = Input((1,), name='user_input')
u = Embedding(num_users, k, name='user_emb')(u_input)
u = Flatten(name='user_flat')(u)
u = Dense(48, activation='relu', name='user_dense')(u)

In [9]:
p_input = Input((1,), name='opening_input')
p = Embedding(num_openings, k, name='opening_emb')(p_input)
p = Flatten(name='opening_flat')(p)
p = Dense(8, activation='relu', name='opening_dense')(p)

In [10]:
x = concatenate([u, p], name='concat')
# x = Dropout(0.1, name='drop1')(x)
x = Dense(16, activation='relu', name='dense1')(x)
x = Dense(4, activation='relu', name='dense2')(x)

In [11]:
u_bias = Embedding(num_users, 1, embeddings_regularizer=l2(l2_lambda), name='user_bias_emb')(u_input)
u_bias = Flatten(name='user_bias_flat')(u_bias)

p_bias = Embedding(num_openings, 1, embeddings_regularizer=l2(l2_lambda), name='opening_bias_emb')(p_input)
p_bias = Flatten(name='opening_bias_flat')(p_bias)

In [12]:
o = concatenate([x, u_bias, p_bias], name='combined_features')
o = Dense(16, activation='relu', name='combined_dense1')(o)
o = Dense(4, activation='relu', name='combined_dense2')(o)
o = Dense(1, activation='linear' , name='output')(o)

In [13]:
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError, RootMeanSquaredError

model = Model(inputs=[u_input, p_input], outputs=o)

metrics = [
    MeanSquaredError(name='mse'),
    MeanAbsoluteError(name='mae'),
    RootMeanSquaredError(name='rmse')
]

model.compile(optimizer=tf.keras.optimizers.legacy.Adam(0.001), loss='mean_squared_error', metrics = metrics)

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 opening_input (InputLayer)  [(None, 1)]                  0         []                            
                                                                                                  
 user_emb (Embedding)        (None, 1, 50)                150       ['user_input[0][0]']          
                                                                                                  
 opening_emb (Embedding)     (None, 1, 50)                150       ['opening_input[0][0]']       
                                                                                              

In [15]:
from keras.utils import plot_model

plot_model(
    model,
    show_shapes=True,
    show_layer_names=True,
    expand_nested=True,
    show_layer_activations=True,
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [16]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.1,
    patience=2,
    verbose=1,
    mode="min",
    baseline=None,
    restore_best_weights=True
)

In [17]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5',monitor='val_loss', verbose=1, save_best_only=True)

In [18]:
history = model.fit(
    x=[X['enc_user_id'], X['enc_opening_id']],
    y=y, epochs=200,
    # validation_data=([X_val['userId'], X_val['movieId']], y_val),
    # callbacks=[early_stopping, model_checkpoint, model_weights_checkpoint]
)

Epoch 1/200
1/1 [==============================] - 0s 265ms/step - loss: 15.9951 - mse: 15.9951 - mae: 3.9994 - rmse: 3.9994
Epoch 2/200
1/1 [==============================] - 0s 3ms/step - loss: 15.9586 - mse: 15.9586 - mae: 3.9948 - rmse: 3.9948
Epoch 3/200
1/1 [==============================] - 0s 3ms/step - loss: 15.9149 - mse: 15.9149 - mae: 3.9893 - rmse: 3.9894
Epoch 4/200
1/1 [==============================] - 0s 3ms/step - loss: 15.8770 - mse: 15.8770 - mae: 3.9846 - rmse: 3.9846
Epoch 5/200
1/1 [==============================] - 0s 3ms/step - loss: 15.8378 - mse: 15.8378 - mae: 3.9797 - rmse: 3.9797
Epoch 6/200
1/1 [==============================] - 0s 3ms/step - loss: 15.7959 - mse: 15.7959 - mae: 3.9744 - rmse: 3.9744
Epoch 7/200
1/1 [==============================] - 0s 3ms/step - loss: 15.7581 - mse: 15.7581 - mae: 3.9696 - rmse: 3.9697
Epoch 8/200
1/1 [==============================] - 0s 3ms/step - loss: 15.7179 - mse: 15.7179 - mae: 3.9646 - rmse: 3.9646
Epoch 9/200
1/

In [19]:
tf.keras.models.save_model(model, '../../../models/openings/recommendations.h5')

/var/folders/8p/bh2hcnvn3bngf4_j4mcz1jx00000gn/T/ipykernel_2958/1383422029.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, '../../../models/openings/recommendations.h5')


In [20]:
user_embeddings = {}

counter = 0

for enc_user_id in df['enc_user_id'].unique():
    user_embedding = model.get_layer('user_emb')(np.array([enc_user_id]))
    user_embedding = tf.keras.backend.flatten(user_embedding)
    user_embedding = tf.expand_dims(user_embedding, axis=0)
    user_dense = model.get_layer('user_dense')(user_embedding)
    
    user_embeddings[user_le.inverse_transform([enc_user_id])[0]] = user_dense.numpy().tolist()

    counter+=1

    if counter%10000==0:
        print(f"Completed {counter} Iterations")

In [21]:
opening_embeddings = {}

counter = 0

for enc_opening_id in df['enc_opening_id'].unique():
    opening_embedding = model.get_layer('opening_emb')(np.array([enc_opening_id]))
    opening_embedding = tf.keras.backend.flatten(opening_embedding)
    opening_embedding = tf.expand_dims(opening_embedding, axis=0)
    opening_dense = model.get_layer('opening_dense')(opening_embedding)
    
    opening_embeddings[opening_le.inverse_transform([enc_opening_id])[0]] = opening_dense.numpy().tolist()

    counter+=1

    if counter%10000==0:
        print(f"Completed {counter} Iterations")

In [22]:
user_bias_embeddings = {}

counter = 0

for enc_user_id in df['enc_user_id'].unique():
    user_bias_embedding = model.get_layer('user_bias_emb')(np.array([enc_user_id]))
    user_bias_embedding = tf.keras.backend.flatten(user_bias_embedding)

    user_bias_embeddings[user_le.inverse_transform([enc_user_id])[0]] = user_bias_embedding.numpy().tolist()

    counter+=1

    if counter%10000==0:
        print(f"Completed {counter} Iterations")

In [23]:
opening_bias_embeddings = {}

counter = 0

for enc_opening_id in df['enc_opening_id'].unique():
    opening_bias_embedding = model.get_layer('opening_bias_emb')(np.array([enc_opening_id]))
    opening_bias_embedding = tf.keras.backend.flatten(opening_bias_embedding)
    
    opening_bias_embeddings[opening_le.inverse_transform([enc_opening_id])[0]] = opening_bias_embedding.numpy().tolist()

    counter+=1

    if counter%10000==0:
        print(f"Completed {counter} Iterations")

In [24]:
import json

with open('../../../models/openings/user_embeddings.json', 'w') as f:
    json.dump(user_embeddings, f)

with open('../../../models/openings/opening_embeddings.json', 'w') as f:
    json.dump(opening_embeddings, f)

with open('../../../models/openings/user_bias_embeddings.json', 'w') as f:
    json.dump(user_bias_embeddings, f)

with open('../../../models/openings/opening_bias_embeddings.json', 'w') as f:
    json.dump(opening_bias_embeddings, f)

In [31]:
with open('../../../models/openings/user_embeddings.json') as f:
    user_embeddings = json.load(f)
with open('../../../models/openings/opening_embeddings.json') as f:
    opening_embeddings = json.load(f)
with open('../../../models/openings/user_bias_embeddings.json') as f:
    user_bias_embeddings = json.load(f)
with open('../../../models/openings/opening_bias_embeddings.json') as f:
    opening_bias_embeddings = json.load(f)

In [26]:
def predict(user_id, opening_id):
    user_embedding = np.array(user_embeddings[user_id])
    opening_embedding = np.array(opening_embeddings[opening_id])

    # Passing user embedding and movie embedding through the concat layer
    concatenated_embeddings = model.get_layer('concat')([user_embedding, opening_embedding])

    # Passing the concatenated embeddings through the dense layers
    x = model.get_layer('dense1')(concatenated_embeddings)
    x = model.get_layer('dense2')(x)

    user_bias_embedding = np.array(user_bias_embeddings[user_id])
    opening_bias_embedding = np.array(opening_bias_embeddings[opening_id])

    user_bias_embedding = user_bias_embedding[:, tf.newaxis]
    opening_bias_embedding = opening_bias_embedding[:, tf.newaxis]

    # Combine embeddings, biases, and pass through the output layer
    input_tensors = [x, user_bias_embedding, opening_bias_embedding]
    concatenated_features = model.get_layer('combined_features')(input_tensors)
    
    x = model.get_layer('combined_dense1')(concatenated_features)
    x = model.get_layer('combined_dense2')(x)
    
    x = model.get_layer('output')(x)

    predicted_rating = x[0][0]

    return predicted_rating.numpy()

In [27]:
def user_recommendations(user_id):
    user_ratings = df[df['user_id'] == user_id]
    user_ratings = user_ratings[user_ratings['score'] != 1]
    
    recommendation = df[~df['opening_id'].isin(user_ratings['opening_id'])][['opening_id']].drop_duplicates()
    recommendation['score_predict'] = recommendation.apply(lambda x: predict(user_id, x['opening_id']), axis=1)
    
    final_rec = recommendation.sort_values(by='score_predict', ascending=False)
    return final_rec

In [28]:
rec = user_recommendations('5f5a90e2-6601-480c-bc08-7592e06beacd')
rec.head()

,opening_id,score_predict
1,1067c159-be01-4f89-a1cf-a32cdd67bb72,3.578439
0,d569ec70-f3fa-421c-88db-0773b26a2475,3.541359


In [29]:
def test_predictions(user_id):
    user_ratings = df[df['user_id'] == user_id]
    
    user_top_ratings = user_ratings.sort_values(by='score', ascending=False)

    comparision = df[df['opening_id'].isin(user_ratings['opening_id'])][['opening_id']].drop_duplicates()
    comparision['score_predict'] = comparision.apply(lambda x: predict(user_id, x['opening_id']), axis=1)
    comparision = user_top_ratings.merge(comparision, on='opening_id')

    return comparision

In [30]:
comparision = test_predictions('5f5a90e2-6601-480c-bc08-7592e06beacd')
comparision

,user_id,opening_id,score,enc_user_id,enc_opening_id,score_predict
0,5f5a90e2-6601-480c-bc08-7592e06beacd,2bc22ae6-ac3d-4aa5-82a4-dd55b8d8709b,4,1,1,3.629077
